In [2]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/data",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /data\t10k-images-idx3-ubyte.gz
Extracting /data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [21]:
element_size=28
time_steps=28
num_classes=10
batch_size=128
hidden_layer_size=128

LOG_DIR="logs/RNN_with_summaries"
_input=tf.placeholder(tf.float32, shape=[None,time_steps,element_size],name='inputs')
y=tf.placeholder(tf.float32, shape=[None,num_classes],name='labels')

batch_x, batch_y=mnist.train.next_batch(batch_size)
batch_x=batch_x.reshape((batch_size,time_steps,element_size))

In [22]:
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean=tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev=tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram',var)
        

In [23]:
with tf.name_scope('rnn_weights'):
    with tf.name_scope("W_x"):
        Wx=tf.Variable(tf.zeros([element_size, hidden_layer_size]))
        variable_summaries(Wx)
    with tf.name_scope("W_h"):
        Wh=tf.Variable(tf.zeros([hidden_layer_size, hidden_layer_size]))
        variable_summaries(Wh)
    with tf.name_scope("Bias"):
        b_rnn=tf.Variable(tf.zeros([hidden_layer_size]))
        variable_summaries(b_rnn)
    

In [24]:
def rnn_step(previous_hidden_state,x):
    current_hidden_state=tf.tanh(tf.matmul(previous_hidden_state,Wh)+tf.matmul(x,Wx)+b_rnn)
    return current_hidden_state

In [25]:
processed_input=tf.transpose(_input,perm=[1,0,2])
initial_hidden=tf.zeros([batch_size, hidden_layer_size])
all_hidden_states=tf.scan(rnn_step,processed_input, initializer=initial_hidden,name='states')

In [26]:
with tf.name_scope('linear_layer_weights') as scope:
    with tf.name_scope("W_linear"):
        Wl=tf.Variable(tf.truncated_normal([hidden_layer_size,num_classes], mean=0, stddev=.01))
        variable_summaries(Wl)
    with tf.name_scope("Bias_linear"):
        bl=tf.Variable(tf.truncated_normal([num_classes],mean=0,stddev=.01))
        variable_summaries(bl)
def get_linear_layer(hidden_state):
    return tf.matmul(hidden_state,Wl)+bl
with tf.name_scope('linear_layer_weights') as scope:
    all_outputs=tf.map_fn(get_linear_layer, all_hidden_states)
    output=all_outputs[-1]
    tf.summary.histogram('outputs',output)

In [27]:
with tf.name_scope('cross_entropy'):
    cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output,labels=y))
    tf.summary.scalar('cross_entropy',cross_entropy)
with tf.name_scope('train'):
    train_step=tf.train.RMSPropOptimizer(0.001,0.9).minimize(cross_entropy)
with tf.name_scope('accuracy'):
    correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(output,1))
    accuracy=(tf.reduce_mean(tf.cast(correct_prediction,tf.float32)))*100
    tf.summary.scalar('accureacy',accuracy)
merged=tf.summary.merge_all()

In [28]:
test_data=mnist.test.images[:batch_size].reshape((-1,time_steps,element_size))
test_label=mnist.test.labels[:batch_size]

In [41]:
with tf.Session() as sess:
    train_writer=tf.summary.FileWriter(LOG_DIR+'/train',graph=tf.get_default_graph())
    test_writer=tf.summary.FileWriter(LOG_DIR+'/test',graph=tf.get_default_graph())
    sess.run(tf.global_variables_initializer())
    for i in range(10000):
        batch_x, batch_y=mnist.train.next_batch(batch_size)
        batch_x=batch_x.reshape((batch_size,time_steps,element_size))
        sess.run([train_step],feed_dict={_input:batch_x,y:batch_y})
        #train_writer.add_summary(summary,i)
        if i%1000==0:
            acc,loss, = sess.run([accuracy,cross_entropy],feed_dict={_input:batch_x,y:batch_y})
            print("Iter "+str(i)+", Minibatch Loss= "+"{:.6f}".format(loss)+", Training Accuracy= "+"{:.5f}".format(acc))
        #if i%10:
            #summary, acc=sess.run([merged,accuracy],feed_dict={_input:test_data, y:test_label})
            #test_writer.add_summary(summary,i)
    test_acc=sess.run(accuracy,feed_dict={_input:test_data, y:test_label})
    print("Test Accuracy:",test_acc)

Iter 0, Minibatch Loss= 2.303116, Training Accuracy= 5.46875
Iter 1000, Minibatch Loss= 1.394797, Training Accuracy= 43.75000
Iter 2000, Minibatch Loss= 0.592246, Training Accuracy= 82.03125
Iter 3000, Minibatch Loss= 0.393879, Training Accuracy= 92.18750
Iter 4000, Minibatch Loss= 0.148196, Training Accuracy= 93.75000
Iter 5000, Minibatch Loss= 0.198646, Training Accuracy= 94.53125
Iter 6000, Minibatch Loss= 0.205712, Training Accuracy= 94.53125
Iter 7000, Minibatch Loss= 0.082582, Training Accuracy= 98.43750
Iter 8000, Minibatch Loss= 0.056843, Training Accuracy= 98.43750
Iter 9000, Minibatch Loss= 0.075027, Training Accuracy= 97.65625
Test Accuracy: 97.65625


In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/data",one_hot=True)
element_size=28
time_steps=28
num_classes=10
batch_size=128
hidden_layer_size=128

_inputs=tf.placeholder(tf.float32, shape=[None,time_steps,element_size],name='inputs')
y=tf.placeholder(tf.float32, shape=[None,num_classes],name='labels')

rnn_cell=tf.contrib.rnn.BasicRNNCell(hidden_layer_size)
outputs,_=tf.nn.dynamic_rnn(rnn_cell,_inputs,dtype=tf.float32)

Wl=tf.Variable(tf.truncated_normal([hidden_layer_size,num_classes], mean=0, stddev=.01))
bl=tf.Variable(tf.truncated_normal([num_classes],mean=0,stddev=.01))

def get_linear_layer(vector):
    return tf.matmul(vector,Wl)+bl

with tf.name_scope('train_basicrnn'):
    last_rnn_output=outputs[:,-1,:]
    final_output=get_linear_layer(last_rnn_output)

    softmax=tf.nn.softmax_cross_entropy_with_logits(logits=final_output,labels=y)
    cross_entropy=tf.reduce_mean(softmax)

    train_step=tf.train.RMSPropOptimizer(0.001,0.9).minimize(cross_entropy)

correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(final_output,1))
accuracy=(tf.reduce_mean(tf.cast(correct_prediction,tf.float32)))*100

sess=tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

test_data=mnist.test.images[:batch_size].reshape((-1,time_steps,element_size))
test_label=mnist.test.labels[:batch_size]

for i in range(3001):
    batch_x, batch_y=mnist.train.next_batch(batch_size)
    batch_x=batch_x.reshape((batch_size,time_steps,element_size))
    sess.run([train_step],feed_dict={_inputs:batch_x,y:batch_y})
    if i%1000==0:
        acc,loss = sess.run([accuracy,cross_entropy],feed_dict={_inputs:batch_x,y:batch_y})
        print("Iter "+str(i)+", Minibatch Loss= "+"{:.6f}".format(loss)+", Training Accuracy= "+"{:.5f}".format(acc))
test_acc=sess.run(accuracy,feed_dict={_inputs:test_data, y:test_label})
print("Test Accuracy:",test_acc)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /data\t10k-images-idx3-ubyte.gz
Extracting /data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Iter 0, Minibatch Loss= 2.303283, Training Accuracy= 7.03125
Iter 1000, Minibatch Loss= 0.144210, Training Accuracy= 96.87500
Iter

In [2]:
import numpy as np
digit_to_word_map={1:"One",2:"Two",3:"Three",4:"Four",5:"Five",6:"Six",7:"Seven",8:"Eight",9:"Nine"}
digit_to_word_map[0]="PAD"
even_sentences=[]
odd_sentences=[]
seqlens=[]
for i in range(10000):
    rand_seq_len=np.random.choice(range(3,7))
    seqlens.append(rand_seq_len)
    rand_odd_ints=np.random.choice(range(1,10,2),rand_seq_len)
    rand_even_ints=np.random.choice(range(2,10,2),rand_seq_len)
    if rand_seq_len<6:
        rand_odd_ints=np.append(rand_odd_ints,[0]*(6-rand_seq_len))
        rand_even_ints=np.append(rand_even_ints,[0]*(6-rand_seq_len))
    even_sentences.append(" ".join([digit_to_word_map[r] for r in rand_odd_ints]))
    odd_sentences.append(" ".join([digit_to_word_map[r] for r in rand_even_ints]))
data=even_sentences+odd_sentences
seqlens*=2
print(even_sentences[0:6])

print(odd_sentences[0:6])
print(data[0:6])
print(seqlens[0:6])

['Three Three Nine Seven PAD PAD', 'Five Five Nine PAD PAD PAD', 'Nine One Nine PAD PAD PAD', 'Three Seven One Seven Nine PAD', 'Seven Three One Nine PAD PAD', 'Three Three Five PAD PAD PAD']
['Six Six Four Six PAD PAD', 'Eight Eight Eight PAD PAD PAD', 'Two Four Two PAD PAD PAD', 'Four Eight Eight Six Four PAD', 'Four Six Eight Two PAD PAD', 'Six Six Six PAD PAD PAD']
['Three Three Nine Seven PAD PAD', 'Five Five Nine PAD PAD PAD', 'Nine One Nine PAD PAD PAD', 'Three Seven One Seven Nine PAD', 'Seven Three One Nine PAD PAD', 'Three Three Five PAD PAD PAD']
[4, 3, 3, 5, 4, 3]


In [3]:
word2index_map={}
index=0
for sent in data:
    for word in sent.lower().split():
        if word not in word2index_map:
            word2index_map[word]=index
            index+=1
index2word_map={index:word for word, index in word2index_map.items()}
vocabulary_size=len(index2word_map)
print(word2index_map)
print(index2word_map)


{'three': 0, 'nine': 1, 'seven': 2, 'pad': 3, 'five': 4, 'one': 5, 'six': 6, 'four': 7, 'eight': 8, 'two': 9}
{0: 'three', 1: 'nine', 2: 'seven', 3: 'pad', 4: 'five', 5: 'one', 6: 'six', 7: 'four', 8: 'eight', 9: 'two'}


In [4]:
dummy=[1]*10000+[0]*10000
labels=[0]*20000
for i in range(len(dummy)):    
    one_hot_encoding=[0,0]
    one_hot_encoding[dummy[i]]=1
    labels[i] = one_hot_encoding
    
data_indices=list(range(len(data)))
np.random.shuffle(data_indices)

data=np.array(data)[data_indices]    
labels=np.array(labels)[data_indices]
seqlens=np.array(seqlens)[data_indices]


train_x=data[:10000]
train_y=labels[:10000]
train_seqlens=seqlens[:10000]

test_x=data[10000:]
test_y=labels[10000:]
test_seqlens=seqlens[10000:]    

In [5]:
def get_sentence_batch(batch_size,data_x,data_y,data_seqlens):
    instance_indices=list(range(len(data_x)))
    np.random.shuffle(instance_indices)
    batch=instance_indices[:batch_size]
    x=[[word2index_map[word] for word in data_x[i].lower().split()] for i in batch]
    y=[data_y[i] for i in batch]
    seqlens=[data_seqlens[i] for i in batch]
    return x,y,seqlens

In [6]:
import LSTM_supervised_embeddings as constval

_inputs=tf.placeholder(tf.int32,shape=[constval.batch_size,constval.times_steps])
_labels=tf.placeholder(tf.float32,shape=[constval.batch_size,constval.num_classes])

_seqlens=tf.placeholder(tf.int32,shape=[constval.batch_size])

In [7]:
with tf.name_scope("embeddings"):
    embeddings=tf.Variable(tf.random_uniform([vocabulary_size,constval.embedding_dimension],-1.0,1.0),name='embedding')
    embed=tf.nn.embedding_lookup(embeddings,_inputs)    

In [8]:
with tf.variable_scope("lstm"):
    lstm_cell=tf.contrib.rnn.BasicLSTMCell(constval.hidden_layer_size,forget_bias=1.0)
    outputs,states=tf.nn.dynamic_rnn(lstm_cell,embed,sequence_length=_seqlens,dtype=tf.float32)
weights={
    'linear_layer':tf.Variable(tf.truncated_normal([constval.hidden_layer_size,constval.num_classes],mean=0,stddev=.01))
}
biases={
    'linear_layer':tf.Variable(tf.truncated_normal([constval.num_classes],mean=0,stddev=.01))
}
final_output=tf.matmul(states[1],weights["linear_layer"])+biases["linear_layer"]
softmax=tf.nn.softmax_cross_entropy_with_logits(logits=final_output,labels=_labels)
cross_entropy=tf.reduce_mean(softmax)

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [9]:
train_step=tf.train.RMSPropOptimizer(0.001,0.9).minimize(cross_entropy)
correct_prediction=tf.equal(tf.argmax(_labels,1),tf.argmax(final_output,1))
accuracy=(tf.reduce_mean(tf.cast(correct_prediction,tf.float32)))*100
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(1000):
        x_batch,y_batch,seqlen_batch=get_sentence_batch(batch_size,train_x,train_y,train_seqlens)
        sess.run(train_step,feed_dict={_inputs:x_batch,_labels:y_batch,_seqlens:seqlen_batch})
        if step % 100==0:
            acc=sess.run(accuracy,feed_dict={_inputs:x_batch,_labels:y_batch,_seqlens:seqlen_batch})
            print("Accuracy at {}:{}".format(step,acc))
    for test_batch in range(5):
        x_test,y_test,seqlen_test=get_sentence_batch(batch_size,test_x,test_y,test_seqlens)
        batch_pred,batch_acc=sess.run([tf.argmax(final_output,1),accuracy],feed_dict={_inputs:x_test,_labels:y_test,_seqlens:seqlen_test})
        print("Test batch accuracy {}:{}".format(test_batch,batch_acc))
    output_example=sess.run([outputs],feed_dict={_inputs:x_test,_labels:y_test,_seqlens:seqlen_test})
    states_example=sess.run([states[1]],feed_dict={_inputs:x_test,_labels:y_test,_seqlens:seqlen_test})

Accuracy at 0:30.46875
Accuracy at 100:100.0
Accuracy at 200:100.0
Accuracy at 300:100.0
Accuracy at 400:100.0
Accuracy at 500:100.0
Accuracy at 600:100.0
Accuracy at 700:100.0
Accuracy at 800:100.0
Accuracy at 900:100.0
Test batch accuracy 0:100.0
Test batch accuracy 1:100.0
Test batch accuracy 2:100.0
Test batch accuracy 3:100.0
Test batch accuracy 4:100.0


In [12]:
print(seqlen_test[1])
print(output_example[0][1].shape)
print(output_example[0][1][:6,0:3])
print(states_example[0][1][0:3])

3
(6, 32)
[[-0.4516028   0.5247337   0.25664377]
 [-0.70696455  0.745164    0.6260608 ]
 [-0.9193043   0.85252273  0.7975621 ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
[-0.9193043   0.85252273  0.7975621 ]
